In [8]:
import pandas as pd
import numpy as np
import sklearn
import math
import scipy
from scipy import stats
from matplotlib import pyplot as plt
import seaborn as sns
from statsmodels.sandbox.stats.multicomp import multipletests

In [3]:
df = pd.read_csv('merged_final.tsv', sep="\t")
df

,eid,SCZ,BD,MDD,DYX,WR,EA,FI_acc,NM_acc,PM_acc,...,SDS_acc,PM_spd,PRM_spd,RT_spd,SDS_spd,TM_num_spd,TM_anum_spd,Speed,Accuracy,General
0,1000015,-0.529286,-1.878759,0.107870,-0.170597,-1.606072,-0.279984,-0.916290,-0.803056,-1.335820,...,NaN,-0.922212,-0.945486,-0.290518,NaN,NaN,NaN,-0.086215,-0.365910,-0.758137
1,1000027,-0.340396,-0.002599,-0.013403,-0.539745,0.907591,2.791107,-0.453247,NaN,-0.182794,...,NaN,-1.798655,0.176377,-1.298758,NaN,NaN,NaN,1.215514,0.460734,-1.074594
2,1000039,1.270266,0.031648,-0.309576,1.304898,-1.502163,0.752067,NaN,NaN,-0.413399,...,NaN,0.170970,NaN,0.645269,NaN,NaN,NaN,-0.317901,-0.179128,0.397982
3,1000040,NaN,NaN,NaN,NaN,NaN,NaN,-1.379332,NaN,-0.644004,...,0.442032,-0.926519,-0.181890,0.630429,0.381423,-0.694125,-0.911270,0.349778,-0.509997,-0.380721
4,1000053,0.622400,1.275362,-0.391936,1.077680,-1.360086,-1.020886,-1.842375,NaN,0.278417,...,NaN,-0.113735,0.520546,-0.290518,NaN,NaN,NaN,-0.003564,-0.632551,-0.520387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498010,6024099,-0.116174,-0.334932,-0.220075,-0.230939,-0.721761,-2.511463,0.472838,NaN,-0.413399,...,1.022833,0.078835,0.550644,0.207821,0.879292,0.643379,0.336162,-0.127401,0.033179,0.769560
498011,6024106,-0.117193,-0.698245,-0.046052,0.569462,-1.733851,-0.354743,NaN,NaN,-1.335820,...,NaN,-1.801678,NaN,-1.393589,NaN,NaN,NaN,0.540108,-0.021402,-1.286910
498012,6024112,-0.298949,-0.002036,-0.894351,1.586845,-0.942153,0.487529,NaN,NaN,0.739627,...,NaN,0.410141,NaN,-0.379857,NaN,NaN,NaN,-0.031236,0.132180,0.045081
498013,6024120,-1.120078,-0.217094,0.520356,0.806120,-0.670318,-0.918365,NaN,NaN,0.970232,...,1.216433,1.518965,NaN,0.319392,1.002620,1.436002,1.427858,-0.392894,-0.009757,1.577710


In [40]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.sandbox.stats.multicomp import multipletests

def compute_correlation_matrix(data, x_cols, y_cols):
    # Initialize results
    results = []
    
    for x in x_cols:
        for y in y_cols:
            # Get common indices where both columns have non-null values
            mask = ~(data[x].isna() | data[y].isna())
            if mask.sum() > 0:  # Only compute if we have valid pairs
                # Get correlation results
                spearman_result = stats.spearmanr(data[x][mask], data[y][mask], nan_policy='omit')
                correlation = spearman_result.correlation
                pvalue = spearman_result.pvalue
                
                results.append({
                    'x': x,
                    'y': y,
                    'correlation': correlation,
                    'pvalue': pvalue,
                    'n': mask.sum()
                })
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    
    # Apply Bonferroni correction using multipletests
    corrected_p = multipletests(results_df['pvalue'], method='fdr_bh')[1]
    results_df['adjusted_p'] = corrected_p
    
    return results_df

# Define column groups in specific order
phenotype_cols = ['SCZ', 'BD', 'MDD', 'DYX', 'WR', 'EA']  # Already in desired order

# Define cognitive measures in specific order
cognitive_cols = [
    # Accuracy measures
    'FI_acc', 'NM_acc', 'PM_acc', 'PRM_acc', 'RT_acc', 'SDS_acc',
    # Speed measures
    'PM_spd', 'PRM_spd', 'RT_spd', 'SDS_spd', 'TM_num_spd', 'TM_anum_spd',
    # Factor scores
    'Speed', 'Accuracy', 'General'
]

# Compute correlations
corr_matrix = compute_correlation_matrix(df, phenotype_cols, cognitive_cols)

# Create correlation and p-value matrices
pivot_corr = corr_matrix.pivot(index='y', columns='x', values='correlation')
pivot_p = corr_matrix.pivot(index='y', columns='x', values='adjusted_p')

# Reorder the indices and columns to match our desired order
pivot_corr = pivot_corr.reindex(index=cognitive_cols, columns=phenotype_cols)
pivot_p = pivot_p.reindex(index=cognitive_cols, columns=phenotype_cols)

# Create the plot
plt.figure(figsize=(12, 10))

# Create heatmap
max_corr = np.nanmax(np.abs(pivot_corr.values))
sns.heatmap(pivot_corr, 
            cmap='RdBu_r',
            center=0,
            vmin=-max_corr,
            vmax=max_corr,
            annot=True,
            fmt='.2f',
            cbar_kws={'label': 'Spearman Correlation'},
            annot_kws={'size': 8})

# Add significance stars
for i in range(pivot_corr.shape[0]):
    for j in range(pivot_corr.shape[1]):
        if not pd.isna(pivot_p.iloc[i, j]):
            if pivot_p.iloc[i, j] < 0.001:
                sig = '***'
            elif pivot_p.iloc[i, j] < 0.01:
                sig = '**'
            elif pivot_p.iloc[i, j] < 0.05:
                sig = '*'
            else:
                sig = ''
                
            if sig:
                plt.text(j + 0.7, i + 0.5, sig,
                        horizontalalignment='left',
                        verticalalignment='center',
                        color='black',
                        fontsize=8)

# Add horizontal lines to separate groups
plt.axhline(y=6, color='black', linewidth=1)  # After accuracy measures
plt.axhline(y=12, color='black', linewidth=1)  # After speed measures

# Customize plot
plt.title('Phenotype-Cognitive Measure Spearman Correlations\n(FDR-corrected)', pad=20)
plt.xlabel('Phenotypes', labelpad=10)
plt.ylabel('Cognitive Measures', labelpad=10)
plt.xticks(rotation=45, ha='right')

# Adjust layout to prevent cutting off labels
plt.tight_layout()

# Save plot
plt.savefig('phenotype_cognitive_correlations_FDR.pdf', bbox_inches='tight', dpi=300)
plt.close()

# Create table of significant correlations
significant_correlations = corr_matrix[corr_matrix['adjusted_p'] < 0.05].copy()
significant_correlations['significance'] = ''
significant_correlations.loc[significant_correlations['adjusted_p'] < 0.05, 'significance'] = '*'
significant_correlations.loc[significant_correlations['adjusted_p'] < 0.01, 'significance'] = '**'
significant_correlations.loc[significant_correlations['adjusted_p'] < 0.001, 'significance'] = '***'

significant_correlations = significant_correlations.rename(columns={
    'x': 'Phenotype',
    'y': 'Cognitive',
    'correlation': 'Spearman_Correlation',
    'pvalue': 'P-value',
    'adjusted_p': 'FDR_P',
    'significance': 'Significance',
    'n': 'N'
})
significant_correlations = significant_correlations.sort_values('FDR_P')

# Save to CSV
significant_correlations.to_csv('significant_correlations_FDR.csv', index=False)

# Print summary statistics
print("\nSummary of Significant Correlations (FDR-corrected P < 0.05):")
print(f"Total number of significant correlations: {len(significant_correlations)}")
print("\nTop 10 strongest correlations by absolute value:")
significant_correlations['abs_correlation'] = abs(significant_correlations['Spearman_Correlation'])
print(significant_correlations.nlargest(10, 'abs_correlation')[
    ['Phenotype', 'Cognitive', 'Spearman_Correlation', 'FDR_P', 'N']
].to_string())


Summary of Significant Correlations (FDR-corrected P < 0.05):
Total number of significant correlations: 87

Top 10 strongest correlations by absolute value:
   Phenotype Cognitive  Spearman_Correlation          FDR_P       N
75        EA    FI_acc              0.211400   0.000000e+00  135535
0        SCZ    FI_acc             -0.170023   0.000000e+00  135535
3        SCZ   PRM_acc             -0.149022   0.000000e+00  140587
14       SCZ   General             -0.137120   0.000000e+00  406414
30       MDD    FI_acc             -0.132089   0.000000e+00  135535
46       DYX    NM_acc             -0.131500  1.554951e-160   41907
45       DYX    FI_acc             -0.124520   0.000000e+00  135535
18        BD   PRM_acc             -0.121260   0.000000e+00  140587
15        BD    FI_acc             -0.120879   0.000000e+00  135535
33       MDD   PRM_acc             -0.117231   0.000000e+00  140587


In [21]:
psychiatric_diagnosis_df = pd.read_csv('data_participant_psych.tsv', sep="\t")
psychiatric_diagnosis_df

,eid
0,4321092
1,5180505
2,5457093
3,4799899
4,4748722
...,...
35916,5718324
35917,4597023
35918,4083151
35919,4518397


In [27]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.multitest import multipletests

def analyze_cognitive_dyslexia(data, psychiatric_eids, cognitive_measures):
    """
    Analyze how DYX PGS predicts cognitive performance in psychiatric cases
    """
    # Create subset of psychiatric cases
    psychiatric_cases = data[data['eid'].isin(psychiatric_eids)].copy()
    
    print(f"\nTotal psychiatric cases: {len(psychiatric_cases)}")
    print(f"Cases with DYX PGS: {psychiatric_cases['DYX'].notna().sum()}")
    
    # Initialize results storage
    results = []
    
    # Analyze each cognitive measure
    for measure in cognitive_measures:
        # Get complete cases for this analysis
        valid_data = psychiatric_cases[[measure, 'DYX']].dropna()
        
        if len(valid_data) < 10:  # Minimum sample size threshold
            print(f"Insufficient data for {measure} (n={len(valid_data)})")
            continue
            
        try:
            # Run regression model
            model = sm.OLS(valid_data[measure], 
                         sm.add_constant(valid_data['DYX'])).fit()
            
            # Store results
            results.append({
                'Cognitive_Measure': measure,
                'Beta': model.params['DYX'],
                'SE': model.bse['DYX'],
                'P_Value': model.pvalues['DYX'],
                'R_Squared': model.rsquared,
                'N': len(valid_data)
            })
            
        except Exception as e:
            print(f"Error analyzing {measure}: {str(e)}")
    
    if not results:
        raise ValueError("No valid results could be computed")
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    
    # Add FDR correction
    results_df['FDR_P'] = multipletests(results_df['P_Value'], method='fdr_bh')[1]
    
    return results_df

def create_visualization(results_df, output_file='dyslexia_cognitive_associations.pdf'):
    """
    Create visualization of associations
    """
    # Create figure
    plt.figure(figsize=(10, 8))
    
    # Define measure groups
    measure_groups = {
        'Accuracy': ['FI_acc', 'NM_acc', 'PM_acc', 'PRM_acc', 'RT_acc', 'SDS_acc'],
        'Speed': ['PM_spd', 'PRM_spd', 'RT_spd', 'SDS_spd', 'TM_num_spd', 'TM_anum_spd'],
        'Factors': ['Speed', 'Accuracy', 'General']
    }
    
    # Add group information
    results_df['Measure_Group'] = 'Other'
    for group, measures in measure_groups.items():
        results_df.loc[results_df['Cognitive_Measure'].isin(measures), 'Measure_Group'] = group
    
    # Sort by measure group and effect size
    results_df['abs_beta'] = abs(results_df['Beta'])
    results_df = results_df.sort_values(['Measure_Group', 'abs_beta'])
    
    # Create forest plot
    y_pos = np.arange(len(results_df))
    
    plt.errorbar(x=results_df['Beta'],
                y=y_pos,
                xerr=results_df['SE'],
                fmt='o',
                capsize=5,
                color='black',
                alpha=0.6)
    
    # Add points colored by group
    for group in measure_groups.keys():
        mask = results_df['Measure_Group'] == group
        plt.scatter(results_df[mask]['Beta'], 
                   y_pos[mask], 
                   label=group,
                   s=100,
                   zorder=5)
    
    # Add reference line at zero
    plt.axvline(x=0, color='black', linestyle='--', alpha=0.5)
    
    # Add significance markers
    for i, row in results_df.iterrows():
        if row['FDR_P'] < 0.001:
            plt.text(row['Beta'], i, '***', ha='left' if row['Beta'] < 0 else 'right', va='center')
        elif row['FDR_P'] < 0.01:
            plt.text(row['Beta'], i, '**', ha='left' if row['Beta'] < 0 else 'right', va='center')
        elif row['FDR_P'] < 0.05:
            plt.text(row['Beta'], i, '*', ha='left' if row['Beta'] < 0 else 'right', va='center')
    
    # Customize plot
    plt.yticks(y_pos, results_df['Cognitive_Measure'])
    plt.xlabel('Beta Coefficient (DYX PGS)', labelpad=10)
    plt.title('Association between Dyslexia PGS and Cognitive Measures\nin Psychiatric Cases', pad=20)
    
    # Add legend
    plt.legend(title='Measure Type', bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Adjust layout and save
    plt.tight_layout()
    plt.savefig(output_file, bbox_inches='tight', dpi=300)
    plt.close()

def main_analysis(df, psychiatric_diagnosis_df):
    """
    Main analysis function
    """
    # Get psychiatric case IDs
    psychiatric_eids = psychiatric_diagnosis_df['eid'].unique()
    
    # Define cognitive measures
    cognitive_measures = [
        # Accuracy measures
        'FI_acc', 'NM_acc', 'PM_acc', 'PRM_acc', 'RT_acc', 'SDS_acc',
        # Speed measures
        'PM_spd', 'PRM_spd', 'RT_spd', 'SDS_spd', 'TM_num_spd', 'TM_anum_spd',
        # Factor scores
        'Speed', 'Accuracy', 'General'
    ]
    
    # Run analysis
    results = analyze_cognitive_dyslexia(df, psychiatric_eids, cognitive_measures)
    
    # Create visualization
    create_visualization(results)
    
    # Save detailed results
    results.to_csv('dyslexia_cognitive_associations.csv', index=False)
    
    # Print summary of significant findings
    print("\nSignificant Associations (FDR < 0.05):")
    significant = results[results['FDR_P'] < 0.05].sort_values('FDR_P')
    if len(significant) > 0:
        print(significant[['Cognitive_Measure', 'Beta', 'P_Value', 'FDR_P', 'N']].to_string())
    else:
        print("No significant associations found")
    
    return results

# Run the analysis
results = main_analysis(df, psychiatric_diagnosis_df)


Total psychiatric cases: 35605
Cases with DYX PGS: 28666

Significant Associations (FDR < 0.05):
   Cognitive_Measure      Beta       P_Value         FDR_P      N
0             FI_acc -0.115054  8.894711e-31  1.334207e-29   9560
13          Accuracy -0.025532  9.260970e-29  6.945727e-28  28616
1             NM_acc -0.133478  4.764730e-12  2.382365e-11   3312
11       TM_anum_spd -0.068085  1.982294e-05  5.946883e-05   4145
14           General -0.019827  1.735347e-05  5.946883e-05  28616
2             PM_acc -0.021035  1.570251e-03  3.925626e-03  28587


In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def create_forest_plot(results_df, output_file='dyslexia_cognitive_associations_revised.pdf'):
    # Define the order of measures
    accuracy_measures = ['FI_acc', 'NM_acc', 'PM_acc', 'PRM_acc', 'RT_acc', 'SDS_acc']
    speed_measures = ['PM_spd', 'PRM_spd', 'RT_spd', 'SDS_spd', 'TM_num_spd', 'TM_anum_spd']
    factor_measures = ['Speed', 'Accuracy', 'General']
    
    # Create measure order
    measure_order = accuracy_measures + speed_measures + factor_measures
    
    # Sort the dataframe
    results_df['Measure_Order'] = pd.Categorical(
        results_df['Cognitive_Measure'], 
        categories=measure_order, 
        ordered=True
    )
    results_df = results_df.sort_values('Measure_Order')
    
    # Create figure
    plt.figure(figsize=(12, 10))
    
    # Plot points and error bars
    y_pos = np.arange(len(results_df))
    
    # Plot error bars
    plt.errorbar(x=results_df['Beta'],
                y=y_pos,
                xerr=results_df['SE'],
                fmt='none',
                capsize=5,
                color='black',
                alpha=0.6,
                zorder=1)
    
    # Add points with different colors for each group
    colors = {'Accuracy': '#1f77b4', 'Speed': '#2ca02c', 'Factors': '#ff7f0e'}
    
    for group, color in colors.items():
        mask = results_df['Measure_Group'] == group
        plt.scatter(results_df[mask]['Beta'], 
                   y_pos[mask], 
                   label=group,
                   color=color,
                   s=100,
                   zorder=2)
    
    # Add vertical line at zero
    plt.axvline(x=0, color='gray', linestyle='--', alpha=0.5, zorder=0)
    
    # Add horizontal lines to separate groups
    plt.axhline(y=5.5, color='black', linestyle='-', alpha=0.3)
    plt.axhline(y=11.5, color='black', linestyle='-', alpha=0.3)
    
    # Customize y-axis labels with red for non-significant results
    y_labels = []
    for _, row in results_df.iterrows():
        if row['FDR_P'] >= 0.05:
            y_labels.append({'label': row['Cognitive_Measure'], 'color': 'red'})
        else:
            y_labels.append({'label': row['Cognitive_Measure'], 'color': 'black'})
    
    plt.yticks(y_pos)
    ax = plt.gca()
    ax.set_yticklabels([item['label'] for item in y_labels])
    for i, tick in enumerate(ax.get_yticklabels()):
        tick.set_color(y_labels[i]['color'])
    
    # Add group labels
    plt.text(-0.25, 2.5, 'Accuracy Measures', rotation=90, va='center', ha='right', fontsize=10, fontweight='bold')
    plt.text(-0.25, 8.5, 'Speed Measures', rotation=90, va='center', ha='right', fontsize=10, fontweight='bold')
    plt.text(-0.25, 13.5, 'Factor Scores', rotation=90, va='center', ha='right', fontsize=10, fontweight='bold')
    
    # Customize plot
    plt.xlabel('Beta Coefficient (DYX PGS)', fontsize=12)
    plt.title('Association between Dyslexia PGS and Cognitive Measures\nin Psychiatric Cases', pad=20, fontsize=14)
    
    # Add legend
    plt.legend(title='Measure Type', bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Add text note about significance
    plt.figtext(0.99, 0.01, 'Red labels indicate non-significant associations (FDR > 0.05)', 
                ha='right', va='bottom', fontsize=8, style='italic')
    
    # Adjust layout and save
    plt.tight_layout()
    plt.savefig(output_file, bbox_inches='tight', dpi=300)
    plt.close()

# Create the plot using the results dataframe
create_forest_plot(results)

In [34]:
cov_df = pd.read_csv('../all_participants_wo_head.tsv', sep="\t", header = None, names = ["eid", "Age", "sex", "PC1", "PC2", "PC3",
                                                                   "PC4", "PC5", "PC6", "PC7", "PC8", "PC9", "PC10"])
cov_df

,eid,Age,sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
0,1000015,66,0.0,-12.68480,2.48984,-0.789333,0.085092,-4.93229,1.051890,-0.627460,-0.334926,-1.76369,0.797701
1,1000053,62,0.0,-11.16330,2.92919,-2.243640,0.952597,-3.33337,-0.932369,-1.366480,0.158433,-2.69394,1.299870
2,1000132,41,0.0,-14.17510,3.03513,-2.020400,-0.546043,-5.84817,-1.924570,0.903956,-0.032164,-3.25403,-1.943640
3,1000148,48,0.0,-9.36496,3.67858,-2.469990,4.969160,4.71670,-1.048090,2.688350,-0.837116,-8.29129,1.841580
4,1000163,42,0.0,-14.23550,3.77995,-1.684350,10.490100,20.03600,1.751440,-0.396306,0.909707,-2.77910,-3.104170
...,...,...,...,...,...,...,...,...,...,...,...,...,...
472571,6023942,61,0.0,-15.50680,2.83479,-2.080370,2.355360,-7.89493,-2.146720,2.006970,-0.999976,-1.45081,0.963057
472572,6024045,64,1.0,-12.02210,3.38163,0.574447,2.077890,-5.38070,1.106630,0.797963,-3.702040,1.52908,6.244980
472573,6024074,62,0.0,-12.82170,2.09487,-1.532360,2.130450,-1.86970,1.635820,0.063636,0.888297,-1.56323,0.310647
472574,6024106,65,0.0,-13.49980,3.66671,-1.412420,1.727630,-2.24667,-2.290800,-0.992355,-0.873021,2.53544,1.824300


In [37]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multitest import multipletests

def analyze_cognitive_dyslexia_with_covariates(data, psychiatric_eids, cognitive_measures, covariates_df):
    """
    Analyze how DYX PGS predicts cognitive performance in psychiatric cases, controlling for covariates
    """
    # Merge main data with covariates
    data_with_cov = data.merge(covariates_df, on='eid', how='inner')
    
    # Create subset of psychiatric cases
    psychiatric_cases = data_with_cov[data_with_cov['eid'].isin(psychiatric_eids)].copy()
    
    print(f"\nTotal psychiatric cases after merging covariates: {len(psychiatric_cases)}")
    print(f"Cases with complete covariate data: {psychiatric_cases.dropna(subset=['Age', 'sex'] + [f'PC{i}' for i in range(1,11)]).shape[0]}")
    
    # Initialize results storage
    results = []
    
    # Covariate columns
    covariate_cols = ['Age', 'sex'] #+ [f'PC{i}' for i in range(1,11)]
    
    # Analyze each cognitive measure
    for measure in cognitive_measures:
        # Get complete cases for this analysis
        valid_data = psychiatric_cases[[measure, 'DYX'] + covariate_cols].dropna()
        
        if len(valid_data) < 10:  # Minimum sample size threshold
            print(f"Insufficient data for {measure} (n={len(valid_data)})")
            continue
            
        try:
            # Prepare predictors (DYX + covariates)
            X = valid_data[['DYX'] + covariate_cols]
            X = sm.add_constant(X)
            y = valid_data[measure]
            
            # Run regression model
            model = sm.OLS(y, X).fit()
            
            # Store results for DYX
            results.append({
                'Cognitive_Measure': measure,
                'Beta': model.params['DYX'],
                'SE': model.bse['DYX'],
                'P_Value': model.pvalues['DYX'],
                'R_Squared': model.rsquared,
                'N': len(valid_data)
            })
            
        except Exception as e:
            print(f"Error analyzing {measure}: {str(e)}")
    
    if not results:
        raise ValueError("No valid results could be computed")
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    
    # Add FDR correction
    results_df['FDR_P'] = multipletests(results_df['P_Value'], method='fdr_bh')[1]
    
    # Add measure grouping
    results_df['Measure_Group'] = 'Other'
    results_df.loc[results_df['Cognitive_Measure'].str.contains('_acc'), 'Measure_Group'] = 'Accuracy'
    results_df.loc[results_df['Cognitive_Measure'].str.contains('_spd'), 'Measure_Group'] = 'Speed'
    results_df.loc[results_df['Cognitive_Measure'].isin(['Speed', 'Accuracy', 'General']), 'Measure_Group'] = 'Factors'
    
    # Add absolute beta for sorting
    results_df['abs_beta'] = abs(results_df['Beta'])
    
    return results_df

def main_analysis(df, psychiatric_diagnosis_df, cov_df):
    """
    Main analysis function with covariates
    """
    # Get psychiatric case IDs
    psychiatric_eids = psychiatric_diagnosis_df['eid'].unique()
    
    # Define cognitive measures
    cognitive_measures = [
        # Accuracy measures
        'FI_acc', 'NM_acc', 'PM_acc', 'PRM_acc', 'RT_acc', 'SDS_acc',
        # Speed measures
        'PM_spd', 'PRM_spd', 'RT_spd', 'SDS_spd', 'TM_num_spd', 'TM_anum_spd',
        # Factor scores
        'Speed', 'Accuracy', 'General'
    ]
    
    # Run analysis with covariates
    results = analyze_cognitive_dyslexia_with_covariates(df, psychiatric_eids, cognitive_measures, cov_df)
    
    # Save detailed results
    results.to_csv('dyslexia_cognitive_associations_with_covariates.csv', index=False)
    
    # Print summary of significant findings
    print("\nSignificant Associations (FDR < 0.05):")
    significant = results[results['FDR_P'] < 0.05].sort_values('FDR_P')
    if len(significant) > 0:
        print(significant[['Cognitive_Measure', 'Beta', 'P_Value', 'FDR_P', 'N', 'R_Squared']].to_string())
    else:
        print("No significant associations found")
    
    return results

# Run the analysis
results = main_analysis(df, psychiatric_diagnosis_df, cov_df)

# Create visualization (using the previously defined function)
create_forest_plot(results, 'dyslexia_cognitive_associations_with_covariates.pdf')


Total psychiatric cases after merging covariates: 33883
Cases with complete covariate data: 32759

Significant Associations (FDR < 0.05):
   Cognitive_Measure      Beta       P_Value         FDR_P      N  R_Squared
0             FI_acc -0.103796  1.889835e-24  2.834753e-23   8942   0.013534
13          Accuracy -0.021094  4.321207e-20  3.240905e-19  26944   0.006446
1             NM_acc -0.122207  1.823054e-10  9.115268e-10   3143   0.016903
11       TM_anum_spd -0.070510  4.181069e-06  1.567901e-05   4008   0.109923
14           General -0.012754  3.385366e-03  1.015610e-02  26944   0.119590
